In [ ]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.7 MB/s eta 0:00:00


In [ ]:
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.6 MB/s eta 0:00:00


In [ ]:
import datasets
pair_ds = datasets.load_dataset('liyongsea/un_linebreak-1000')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/668128 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/84617 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/liyongsea___parquet/liyongsea--un_linebreak-1000-bb158f3f17c3a309/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pair_ds['train'][204]

{'text': 'of poverty eradication, peace and development;\n4. Acknowledges the findings of the midterm comprehensive global review,',
 'label': True}

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    # return tokenizer(examples["text"], padding="max_length", truncation=True)
    # return tokenizer(examples["text"], padding="longest", truncation=True, max_length=128)
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)

In [ ]:
tokenizer("Hi! This is Jia"), tokenizer("Hi! this is Jia")

({'input_ids': [101, 8790, 106, 1188, 1110, 13997, 1161, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [101, 8790, 106, 1142, 1110, 13997, 1161, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]})

In [ ]:
tokenized_datasets = pair_ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/668128 [00:00<?, ? examples/s]

Map:   0%|          | 0/84617 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
len(tokenized_datasets['test'][0]['input_ids'])

64

In [ ]:
tokenized_datasets["train"]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 668128
})

In [ ]:
small_train_dataset = tokenized_datasets["train"].select(range(100000))
small_eval_dataset = tokenized_datasets["test"].select(range(2000))


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=3,
    save_strategy="epoch", weight_decay=0.01,
  )

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.234000,0.433054,0.862000
2,0.172100,0.530246,0.855500
3,0.123100,0.612475,0.843500


TrainOutput(global_step=37500, training_loss=0.19051195841471355, metrics={'train_runtime': 2403.5168, 'train_samples_per_second': 124.817, 'train_steps_per_second': 15.602, 'total_flos': 4967527449600000.0, 'train_loss': 0.19051195841471355, 'epoch': 3.0})

In [ ]:
small_eval_dataset = tokenized_datasets["test"].select(range(2000))

In [ ]:
predictions = trainer.predict(small_eval_dataset)

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
for i, (y_pred, y_true) in enumerate(zip(preds, predictions.label_ids)):
    if y_pred == 0 and y_true == 1:
      print(i, small_eval_dataset[i]['label'], repr(small_eval_dataset[i]['text']))

3 True '14-28038 (E) 040414\n*1428038*'
16 True 'Development Goals for women and girls\nChallenges and achievements in the implementation of the'
37 True 'Millennium Development Goals for women and girls.\nProgress and challenges in achieving the Millennium DevelopmentGoals for'
80 True 'A human rights-based approach to development\n7. Many participants agreed that a human rights perspective was necessary, both'
109 True 'equality must be comprehensively implemented at all levels.\nResource allocation, gender-responsive budgeting and social protection'
156 True 'Lack of sex-disaggregated data and a focus on data collection\n15. Gender statistics were essential to inform policy formulation and to hold actors'
190 True 'post-2015 development agenda.\nA stand-alone goal on gender equality and gender mainstreaming'
191 True 'A stand-alone goal on gender equality and gender mainstreaming\n19. Participants expressed wide support for a stand-alone goal on gender equality'
251 True 'Venezuela 